In [1]:
###############################################################################################
############################################ FUNCIONES ########################################
###############################################################################################

In [2]:
import numpy as np
import copy

In [3]:
def crea_matriz(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias
                            
    params: renglones       no. de renglones de la matriz
            columnas        no. de renglones de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

In [4]:
def house(x):
    """
    Función que calcula la proyección de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    constante utilizada para obtener v
            v       vector que representa la reflexión de householder
    """
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

In [5]:
def factorizacion_QR(A):
    """
    Función que genera una matriz que contendrá información escencial de las proyecciones householder
    (vectores v's) y componentes de la matriz triangular superior R, del estilo:
    [r11      r12      r13      r14    ]
    [v_2_(1)  r22      r23      r24    ]
    [v_3_(1)  v_3_(2)  r33      r34    ]
    [v_4_(1)  v_4_(2)  v_4_(3)  r44    ]
    [v_5_(1)  v_5_(2)  v_5_(3)  v_5_(4)]
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: A_r_v  Matriz (mxn) con la información escencial (es igual a la matriz R, pero en lugar de tener ceros
                   en la parte inferior, contiene info de los vectores householder que serán útiles para
                   futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)

    """
    m=A.shape[0]
    n=A.shape[1]
    A_r_v=copy.copy(A)
    for j in range(n):
        beta, v=house(A_r_v[j:m,j])
        A_r_v[j:m,j:n]=A_r_v[j:m,j:n]-beta*(np.outer(v,v)@A_r_v[j:m,j:n])
        A_r_v[(j+1):m,j]=v[1:(m-j)]
    return A_r_v

In [6]:
def QT_C(A_r_v,C):
    """
    Función que calcula el producto matricial de Q_transpuesta por una matriz dada C
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: M       Matriz con numero al azar
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    QT_por_C=np.eye(m)
    for j in range(n-1,-1,-1):
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        QT_por_C[j:m,j:m]=C[j:m,j:m]-beta*np.outer(v,v)@C[j:m,j:m]
    return QT_por_C

In [7]:
def Q_j(A_r_v,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van obteniendo n Qj's,
    que si se multiplican todas da por resultado Q=Q1*Q2*...*Qn)
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Qj=np.eye(m)
    v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
    beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
    Qj[j:m,j:m]=np.eye(m-j)-beta*np.outer(v,v)
    return Qj

In [8]:
def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """
    #FALTA CONTEMPLAR POSIBLES PROBLEMAS/SOLUCIONES PARA LOS DIFERENTES TIPOS DE SEL (SIN SOLUCION, MULTIPLES SOLUCIONES,
    #SOLUCION ÚNICA). INCORPORAR ESTO PARA EVITAR ERRORES FATALES/DESBORDAMIENTO/ETC
    m=A_r_v.shape[0]
    A_r_v=factorizacion_QR(A)
    Q=np.transpose(QT_C(A_r_v,np.eye(m)))
    R=np.transpose(Q)@A
    b_prima=np.transpose(Q)@b
    x=Solucion_SHA_triangula_nxn(R,b_prima)
    return x

In [ ]:
def Solucion_SHA_triangular_superior_nxn(R,b_prima):
    #ALGORITMO SIMPLE DE SUSTITUCION HACIA ATRAS CON UNA MATRIZ TRIANGULAR SUPERIOR (SOLO HAY QUE SUSTITUIR ITERATIVAMENTE)
    return x

In [9]:
###############################################################################################
##################################### PRUEBA DE FUNCIONES #####################################
###############################################################################################

In [10]:
m=5
n=3
A=np.round(crea_matriz(m,n,6,-6,False),2)

#Prueba también definiendo una matriz entrada por entrada, solo asegurate que sus entradas sean
#tipo flotantes/dobles (pues si son de tipo enteras arrastrarán errores de redondeo considerables)
# m=3
# n=3
#A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')

In [11]:
A_r_v=factorizacion_QR(A)
np.round(A_r_v,4)

array([[ 6.8903, -1.805 ,  3.4823],
       [ 0.5446,  6.1372,  0.8945],
       [-0.4461, -0.5083,  6.1648],
       [ 0.5769,  0.1314,  0.1671],
       [ 0.54  , -1.2968,  0.8475]])

In [12]:
#Obtención de Q con la función QT_C
Q=np.transpose(QT_C(A_r_v,np.eye(m)))
np.round(Q,4)

array([[ 0.0566, -0.5138,  0.4209, -0.5442, -0.5094],
       [-0.5138,  0.7202,  0.2292, -0.2964, -0.2774],
       [ 0.4209,  0.2292,  0.8122,  0.2428,  0.2273],
       [-0.5442, -0.2964,  0.2428,  0.686 , -0.2939],
       [-0.5094, -0.2774,  0.2273, -0.2939,  0.7249]])

In [13]:
#Así obtenemos R
R=np.transpose(Q)@A
np.round(R,4)

array([[ 6.8903, -1.805 ,  3.4823],
       [-0.    ,  1.9865, -5.1621],
       [-0.    ,  2.1096,  2.1819],
       [ 0.    , -0.5452, -1.9752],
       [ 0.    ,  5.3825,  1.8698]])

In [14]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
Q=np.eye(m)
R=copy.copy(A)
for j in range(n):
    Qj=Q_j(A_r_v,j)
    print('\nQ',j,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=Q_j(A_r_v,j)@R
    print('\nAplicando las Qjs a A por la izquierda para obtener R, iteracion ',j,':\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))



Q 0 :
 [[ 0.0566 -0.5138  0.4209 -0.5442 -0.5094]
 [-0.5138  0.7202  0.2292 -0.2964 -0.2774]
 [ 0.4209  0.2292  0.8122  0.2428  0.2273]
 [-0.5442 -0.2964  0.2428  0.686  -0.2939]
 [-0.5094 -0.2774  0.2273 -0.2939  0.7249]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  0 :
 [[ 6.8903 -1.805   3.4823]
 [-0.      1.9865 -5.1621]
 [-0.      2.1096  2.1819]
 [ 0.     -0.5452 -1.9752]
 [ 0.      5.3825  1.8698]]

Q 1 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      0.3237  0.3437 -0.0888  0.877 ]
 [ 0.      0.3437  0.8253  0.0452 -0.4458]
 [ 0.     -0.0888  0.0452  0.9883  0.1152]
 [ 0.      0.877  -0.4458  0.1152 -0.1373]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  1 :
 [[ 6.8903 -1.805   3.4823]
 [ 0.      6.1372  0.8945]
 [-0.     -0.     -0.8964]
 [ 0.     -0.     -1.1796]
 [ 0.     -0.     -5.9841]]

Q 2 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.    ]
 [ 0.      0.     -0.1454 -0.1913 -0.9707]

In [15]:
##################################### FUNCION DESECHADA #####################################

# def Factorizacion_Q_R(A):
#     m=A.shape[0]
#     n=A.shape[1]
#     R=copy.copy(A)
#     Q=np.eye(m)
#     for j in range(n):
#         Qj=np.eye(m)
#         beta, v=house(R[j:m,j])
#         P=np.eye(m-j)-beta*(np.outer(v,v))
#         R[j:m,j:n]=P@R[j:m,j:n]
#         Qj[j:m,j:m]=P
#         Q=Qj@Q
#     Q=np.transpose(Q)
#     return Q,R


# Q,R=Factorizacion_Q_R(A)
# print('\n\nQ:\n',np.round(Q,4))
# print('\nR:\n',np.round(R,4))
# print('\nQ@R:\n',np.round(Q@R,4))
# print('\nA:\n',A)

In [16]:
#Prubemos un ejemplo más grande con 10^4 entradas
m=125
n=80
A=np.round(crea_matriz(m,n,10,-10,False),2)
A_r_v=factorizacion_QR(A)
A_r_v

array([[ 6.57258458e+01, -9.02900515e+00, -5.51529000e+00, ...,
         3.68092639e+00,  5.04291875e+00, -4.67291667e+00],
       [ 1.06189872e-01,  6.28972660e+01, -1.71800761e+00, ...,
        -1.65263617e+00,  8.67220428e+00,  2.57170497e+00],
       [ 2.84800422e-03,  4.52402847e-02,  6.22779128e+01, ...,
         9.98059676e+00, -3.18247865e+00, -1.26995984e+01],
       ...,
       [ 9.71033820e-02, -9.83803022e-02,  2.61975378e-02, ...,
         2.24952874e-01,  2.13025970e-02,  1.30474492e-01],
       [-9.07292774e-02,  8.91852938e-02, -1.09253068e-01, ...,
         1.61016740e-01,  2.60233535e-02,  3.03451385e-02],
       [ 2.56320380e-02, -9.50611807e-03, -5.57806922e-02, ...,
        -5.19127394e-02,  3.90623445e-02,  1.05213578e-01]])